In [47]:
import os
import warnings
import joblib
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score, accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed 
from sklearn import ensemble, linear_model, svm
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from lazy_test import *
from success_utils import *

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Create a dictionary to group countries by region
region_groups = {
    "North America": ["HI", "Mexico", "NV", "OK", "NM","CA", "NY", "MD", "DC", "WA", "CO", "MN", "TX", "SC", "GA", "KY", "PA", "MI", "OH", "OR", "NC", "IL", "AZ", "TN", "NJ", "UT", "VA", "ND", "VT", "IA", "ND", "VT", "LA", "CT", "RI"],
    "Europe": ["Martinique", "AL", "Cayman Islands", "Russia", "ME", "UK", "DE", "Poland", "Denmark", "Netherlands", "Norway", "Liechtenstein", "Austria", "Belgium", "Czech Republic", "Iceland", "Ireland", "France", "Spain", "Sweden", "Switzerland", "Italy", "Slovenia", "Greece"],
    "Asia": ["MO", "IN", "Japan", "China", "Hong Kong", "India", "Viet Nam", "Taiwan", "Afghanistan", "Israel","Sri Lanka", "Jordan"],
    "South America": ["Colombia", "Brazil"],
    "Africa": ["WI", "SD", "MA", "Ghana", "Ethiopia", "Liberia", "Rwanda"],
    "Oceania": ["AU", "NZ"],
    "Other": [ "United Arab Emirates", "Svalbard and Jan Mayen"]
}

noticable_entities = ['ORG', 'PERSON', 'DATE',
                        'CARDINAL', 'GPE', 'PRODUCT', 
                        'WORK_OF_ART','ORDINAL', 'MONEY',
                        'TIME', 'NORP']


data_path = '/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Data/Kickstarter_only/final_data.csv'
success_data = pd.read_csv(data_path)
success_data['collection_ratio'] = success_data.collected.apply(decode_amount)/success_data.goal_amount.apply(decode_amount)


bins = [0, 7, 14, 30, 60]
labels = ['<1 week', '1-2 weeks', '2 weeks - 1 month', '1-2 months']
success_data['duration'] = pd.cut(success_data['duration'], bins=bins, labels=labels)
success_data= success_data[success_data.donate != 1].reset_index(drop=True)


success_weights = (1-success_data.success.value_counts()/len(success_data)).values


def get_region(country):
    for region, countries in region_groups.items():
        if country in countries:
            return region
    return "Other"

success_data['region'] = success_data.country.str.strip().apply(get_region)
success_data.num_backers = success_data.num_backers.str.replace(',', '').astype(int)
success_data = success_data[success_data.region == 'North America']

success_data = pd.get_dummies(
                    success_data,
                    columns=['fb_type', 
                              'duration'])


for entity in noticable_entities:
    success_data[f'fb_entity_{entity}'] = success_data.fb_entities_identified.fillna('None').str.split(',').apply(lambda entity_list: entity in entity_list).astype(int)

bool_cols = success_data.dtypes[success_data.dtypes == 'bool'].index.tolist()
success_data[bool_cols] = success_data[bool_cols].astype(int)

#ecoding the number of backers
success_data.num_backers = encode_targets(success_data.num_backers)
success_data.population = np.exp(success_data.population)


success_data.head()

,collected,goal_amount,num_backers,category,pledge_types,donate,project_we_love,success,start_month,start_day,population,country,Person,Team,num_projects,num_backed,verified,fb_page_name,fb_likes_at_posting,fb_followers_at_posting,fb_likes,fb_comments,fb_shares,fb_post_views,fb_readability,fb_readability_lix,fb_entropy_scores,fb_perplexity_scores,fb_fear,fb_anger,fb_anticip,fb_trust,fb_surprise,fb_positive,fb_negative,fb_sadness,fb_disgust,fb_joy,fb_entities_identified,fb_post_sponsored,fb_post_age,fb_page_age,fb_positive_reactions,fb_negative_reactions,fb_topic_0,fb_topic_1,fb_topic_2,fb_topic_3,fb_topic_4,fb_topic_5,fb_topic_6,fb_topic_7,fb_topic_8,fb_topic_9,fb_topic_10,collection_ratio,region,fb_type_link,fb_type_photo,fb_type_video,duration_<1 week,duration_1-2 weeks,duration_2 weeks - 1 month,duration_1-2 months,fb_entity_ORG,fb_entity_PERSON,fb_entity_DATE,fb_entity_CARDINAL,fb_entity_GPE,fb_entity_PRODUCT,fb_entity_WORK_OF_ART,fb_entity_ORDINAL,fb_entity_MONEY,fb_entity_TIME,fb_entity_NORP
2,12.960635,10.819778,6.823286,design,4.0,0.0,1.0,1,6,1,684451.0,WA,0,1,7,16,Marc Barros,Kickstarter,1589901,1588019,10,2,1,12,4.836818,33.727273,0.719996,2.054425,0.000024,0.000011,0.000037,0.000062,0.000093,1.000000,0.000029,0.000020,0.000028,0.000062,PRODUCT,0,2,4839,2,0,0.000064,0.000036,0.000074,0.242741,0.000100,0.000050,0.450027,0.075764,0.085070,0.000038,0.116083,8.506720,North America,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4,11.866416,9.903488,6.954639,comics,8.0,0.0,1.0,1,6,1,105319.0,CA,1,0,3,88,Adam Lawson,Kickstarter,1589930,1588019,4,1,0,5,6.933333,29.111111,0.625279,1.868768,0.000057,0.000018,0.000070,0.000098,0.000096,0.000032,0.000036,0.000013,0.000020,0.000074,"CARDINAL,PERSON",0,8,4839,0,1,0.000026,0.000060,0.939369,0.000080,0.000065,0.000032,0.000011,0.000050,0.000082,0.000096,0.000015,7.120150,North America,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0
5,10.374272,10.308953,5.318120,film_Video,13.0,0.0,1.0,1,6,1,8175133.0,NY,1,0,1,2,Julia Greenberg,Kickstarter,1589715,1587803,3,1,0,4,15.900000,61.666667,0.284582,1.329206,0.285714,0.000047,0.000050,0.000100,0.000088,0.285714,0.142857,0.000096,0.000082,0.142857,"PERSON,GPE",0,9,4839,0,1,0.000019,0.154496,0.000084,0.081168,0.000079,0.126198,0.000052,0.156902,0.294644,0.163853,0.000095,1.067500,North America,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0
6,9.542446,8.804775,5.556828,comics,16.0,0.0,1.0,1,6,1,3971883.0,CA,0,1,4,23,Donold Wooldridge,Kickstarter,1589715,1587803,3,5,1,5,14.239500,45.000000,0.570874,1.769813,0.200000,0.200000,0.000072,0.000028,0.000076,0.000049,0.400000,0.200000,0.000072,0.000036,WORK_OF_ART,0,10,4839,1,1,0.000043,0.000028,0.162778,0.000097,0.000045,0.000027,0.059113,0.062181,0.060710,0.626484,0.000060,2.091059,North America,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
8,11.346304,9.210340,7.051856,games,27.0,0.0,1.0,1,6,4,621849.0,MD,1,0,5,21,Amanda Coronado,Kickstarter,1589543,1587618,9,3,1,11,3.083867,20.333333,0.466686,1.594701,0.000036,0.000051,0.000068,0.000010,0.000083,1.000000,0.000015,0.000030,0.000068,0.000067,"PERSON,CARDINAL",0,14,4839,1,1,0.000018,0.000018,0.073925,0.798958,0.000098,0.000041,0.000093,0.000046,0.000022,0.000042,0.102862,8.465200,North America,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0


In [48]:
dropping_columns = ['collected', 'verified', 'donate', 'project_we_love', 'Team' ,'country', 'fb_page_name', 'region', 'fb_entities_identified', 'population', 'category']

success_data.drop(columns=dropping_columns, axis=1, inplace=True)
success_data = success_data[success_data.collection_ratio < 10].reset_index(drop=True)
success_data.pledge_types = success_data.pledge_types.astype(int)
success_data.num_projects = success_data.num_projects.astype(int)
success_data.num_backed = success_data.num_backed.astype(int)


In [49]:
#identify and list all the boolean features disguised as int in the dataset
bool_features = []
for col in success_data.columns:
    if success_data[col].nunique() == 2:
        bool_features.append(col)

bool_features.remove('success')

In [50]:
entity_features = [x for x in bool_features if 'fb_entity' in x]
duration_features = [x for x in bool_features if 'duration' in x]
region_features = [x for x in bool_features if 'region' in x]


In [51]:
# from sklearn.decomposition import PCA, TruncatedSVD
# def save_embedding_info(loadings, entity_features, name):
#     # Create a dataframe to store the loadings
#     loadings_df = pd.DataFrame(loadings, columns=entity_features)

#     plt.figure(figsize=(20, 10))
#     sns.heatmap(loadings_df, cmap='RdBu', center=0, annot=True)
#     save_path = '/workspaces/Crowdfunding-Social-Media-Drivers/Modelling/02_success_engagement/heatmaps for embeddings'
#     plt.savefig(save_path + f'/{name}.png', bbox_inches='tight')

# def try_svd(df, cols):
#     for i in range(1, len(cols)):
#         svd = TruncatedSVD(n_components=i)
#         svd.fit(df[cols])
#         evr = svd.explained_variance_ratio_.sum()
#         print(f'Explained variance ratio for {i} components: {svd.explained_variance_ratio_.sum()}')
#         if evr > 0.8:
#             return svd, i

# def append_embeddings(embeddings, df, col):
#     for i in range(embeddings.shape[1]):
#         df[f'{col}_embedding_{i}'] = embeddings[:,i]
#     return df

# svd, n_components = try_svd(success_data, entity_features)
# embddings = svd.fit_transform(success_data[entity_features])
# loadings = svd.components_
# save_embedding_info(loadings, entity_features, 'fb_entities')

# success_data = append_embeddings(embddings, success_data, 'fb_entities')

# svd, n_components = try_svd(success_data, region_features)
# embddings = svd.fit_transform(success_data[region_features])
# loadings = svd.components_
# save_embedding_info(loadings, region_features, 'region')

# success_data = append_embeddings(embddings, success_data, 'region')

# svd, n_components = try_svd(success_data, categories)
# embddings = svd.fit_transform(success_data[categories])
# loadings = svd.components_
# save_embedding_info(loadings, categories, 'categories')

# success_data = append_embeddings(embddings, success_data, 'categories')

# success_data.drop(columns=entity_features + region_features + categories, axis=1, inplace=True)

In [52]:
success_data.num_backers = np.exp(success_data.num_backers)
target_cols = [
    'success',
    'collection_ratio',
    'num_backers'
]
emotion_cols = ['fb_fear','fb_anger', 'fb_anticip', 'fb_trust', 'fb_surprise', 'fb_positive', 'fb_negative', 'fb_sadness', 'fb_disgust', 'fb_joy']
success_data['fb_emotionality'] = success_data[emotion_cols].sum(axis=1)

fb_cols = success_data.columns[success_data.columns.str.startswith('fb')].tolist()
common_features= success_data.columns[~success_data.columns.str.startswith('fb')].tolist()
common_features = [feature for feature in common_features if feature not in target_cols]

engagement_features = [ 'fb_likes', 'fb_comments', 'fb_shares','fb_positive_reactions','fb_negative_reactions']
post_fb_features = [feature for feature in fb_cols if feature not in engagement_features]


In [53]:
domain_page_features = success_data[common_features]
domain_engagement_features = success_data[common_features + engagement_features]
domain_post_features = success_data[common_features + post_fb_features]

targets = success_data[target_cols]

In [54]:
pd.concat([domain_page_features, targets], axis=1).to_csv('/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Data/final_datasets/domain_only.csv', index=False)
pd.concat([domain_engagement_features, targets], axis=1).to_csv('/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Data/final_datasets/domain_engagement.csv', index=False)
pd.concat([domain_post_features, targets], axis=1).to_csv('/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Data/final_datasets/domain_post.csv', index=False)

In [55]:
print(common_features)

['goal_amount', 'pledge_types', 'start_month', 'start_day', 'Person', 'num_projects', 'num_backed', 'duration_<1 week', 'duration_1-2 weeks', 'duration_2 weeks - 1 month', 'duration_1-2 months']


## Modelling

In [56]:
working_data = 'domain_engagement'
domain_post = pd.read_csv(f'/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Data/final_datasets/{working_data}.csv')

In [57]:
domain_post.loc[:,target_cols].head()

,success,collection_ratio,num_backers
0,1,8.506720,919.0
1,1,7.120150,1048.0
2,1,1.067500,204.0
3,1,2.091059,259.0
4,1,8.465200,1155.0


In [58]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(domain_post.drop(target_cols, axis=1), domain_post[target_cols], test_size=0.15, random_state=42, stratify=domain_post.success)

X_train = X_train.reset_index(drop=True).values
X_test = X_test.reset_index(drop=True).values
y_train = y_train.reset_index(drop=True).values
y_test = y_test.reset_index(drop=True).values

* y[:, 0] => Success (Boolean)
* y[:,1] => Collection Ratio
* y[:,2] => Number of Backers


In [59]:
reg_test = RegressionModels(X_train, X_test, y_train[:,1], y_test[:,1], decode=False)
reg_test.run_evaluation()
print(reg_test.not_trained_)
print(reg_test.not_evaluated_)
reg_test.top_10_socres.to_csv("/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Results/03 - Domain Engagement/collection_ratio/lazy_scores.csv")
reg_test.top_10_socres

Evaluating models: 100%|██████████| 40/40 [00:00<00:00, 138.54it/s]

All models were trained successfully.
All models were evaluated successfully.


,Insample RMSE,Insample MAE,Insample R2,Insample EV Score,Outsample RMSE,Outsample MAE,Outsample R2,Outsample EV Score
RandomForestRegressor,0.744,0.492,0.860,0.860,1.772,1.226,0.094,0.094
Lasso,1.917,1.298,0.067,0.067,1.776,1.265,0.091,0.095
LassoLars,1.917,1.298,0.067,0.067,1.776,1.265,0.091,0.095
ElasticNet,1.904,1.281,0.081,0.081,1.779,1.253,0.087,0.091
BayesianRidge,1.899,1.275,0.085,0.085,1.786,1.255,0.080,0.085
OrthogonalMatchingPursuitCV,1.936,1.317,0.049,0.049,1.788,1.282,0.078,0.082
OrthogonalMatchingPursuit,1.936,1.317,0.049,0.049,1.788,1.282,0.078,0.082
ElasticNetCV,1.882,1.255,0.102,0.102,1.789,1.251,0.077,0.080
LassoCV,1.881,1.254,0.102,0.102,1.789,1.252,0.077,0.080
LarsCV,1.881,1.254,0.103,0.103,1.790,1.253,0.076,0.079


In [60]:
reg_test = RegressionModels(X_train, X_test, y_train[:,2], y_test[:,2], decode=False)
reg_test.run_evaluation()
print(reg_test.not_trained_)
print(reg_test.not_evaluated_)
reg_test.top_10_socres.to_csv("/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Results/03 - Domain Engagement/num_backers/lazy_scores.csv")
reg_test.top_10_socres

Evaluating models: 100%|██████████| 40/40 [00:00<00:00, 127.22it/s]

All models were trained successfully.
All models were evaluated successfully.


,Insample RMSE,Insample MAE,Insample R2,Insample EV Score,Outsample RMSE,Outsample MAE,Outsample R2,Outsample EV Score
GradientBoostingRegressor,600.234,339.008,0.920,0.920,2381.774,906.305,0.450,0.451
RandomForestRegressor,637.779,256.259,0.910,0.910,2470.909,956.475,0.408,0.408
XGBRegressor,10.779,6.878,1.000,1.000,2627.020,975.051,0.331,0.335
PoissonRegressor,1728.229,735.600,0.338,0.338,2641.296,1027.980,0.323,0.345
Lars,1904.853,918.676,0.196,0.196,2682.288,1113.223,0.302,0.317
LinearRegression,1899.613,860.947,0.200,0.200,2731.763,1081.053,0.276,0.292
Ridge,1899.618,860.248,0.200,0.200,2732.549,1080.856,0.276,0.291
LassoLars,1899.649,860.429,0.200,0.200,2732.644,1081.803,0.276,0.291
Lasso,1899.649,860.429,0.200,0.200,2732.645,1081.802,0.276,0.291
HistGradientBoostingRegressor,1468.415,409.144,0.522,0.522,2736.883,1037.050,0.273,0.282


In [61]:
cls_test = ClassificationModels(X_train, X_test, y_train[:,0], y_test[:,0])
cls_test.run_evaluation()
print(cls_test.not_trained_)
print(cls_test.not_evaluated_)
cls_test.top_10_socres.to_csv("/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Results/03 - Domain Engagement/success/lazy_scores.csv")
cls_test.top_10_socres

Evaluating models: 100%|██████████| 27/27 [00:00<00:00, 154.30it/s]

The modelsNu-Support Vector Machine could not be trained.
The modelsKernel Ridge could not be evaluated.


,Accuracy,Precision,Recall,F1 Score,ROC AUC,Class-0 Acc,Class-1 Acc
Gaussian Naive Bayes,0.301205,1.000000,0.226667,0.369565,0.613333,1.000,0.226667
Easy Ensemble Classifier,0.626506,0.958333,0.613333,0.747967,0.681667,0.750,0.613333
Linear Support Vector Machine,0.180723,0.818182,0.120000,0.209302,0.435000,0.750,0.120000
Multinomial Naive Bayes,0.554217,0.931818,0.546667,0.689076,0.585833,0.625,0.546667
Balanced Random Forest Classifier,0.698795,0.931034,0.720000,0.812030,0.610000,0.500,0.720000
RUSBoost Classifier,0.746988,0.897059,0.813333,0.853147,0.469167,0.125,0.813333
Balanced Bagging Classifier,0.674699,0.887097,0.733333,0.802920,0.429167,0.125,0.733333
Gaussian Process,0.807229,0.904110,0.880000,0.891892,0.502500,0.125,0.880000
Decision Tree,0.855422,0.909091,0.933333,0.921053,0.529167,0.125,0.933333
XGBoost,0.891566,0.902439,0.986667,0.942675,0.493333,0.000,0.986667


#### Classification Notes
> For the success model, based on the overall f1 scores, Balance RandomForest clssifier, Balanced Bagging Classifier are more suitable

## Further Finetuning of the topk models

### Collection Ratio

In [24]:
lars = linear_model.Lars(random_state=42)
lars.fit(X_train, y_train[:,1])

in_y_pred = lars.predict(X_train)
out_y_pred = lars.predict(X_test)

print(f'In-sample RMSE, R2: {np.sqrt(mean_squared_error(y_train[:,1], in_y_pred)):.2f}, {r2_score(y_train[:,1], in_y_pred)}')
print(f'Out-sample RMSE, R2: {np.sqrt(mean_squared_error(y_test[:,1], out_y_pred)):.2f}, {r2_score(y_test[:,1], out_y_pred)}')


In-sample RMSE, R2: 1.87, 0.11134964209032583
Out-sample RMSE, R2: 1.83, 0.03228931724211459


In [25]:
lassolars = linear_model.LassoLars(random_state=42)
lassolars.fit(X_train, y_train[:,1])

in_y_pred = lassolars.predict(X_train)
out_y_pred = lassolars.predict(X_test)

print(f'In-sample RMSE, R2: {np.sqrt(mean_squared_error(y_train[:,1], in_y_pred)):.2f}, {r2_score(y_train[:,1], in_y_pred)}')
print(f'Out-sample RMSE, R2: {np.sqrt(mean_squared_error(y_test[:,1], out_y_pred)):.2f}, {r2_score(y_test[:,1], out_y_pred)}')

In-sample RMSE, R2: 1.92, 0.06746576616619737
Out-sample RMSE, R2: 1.78, 0.09056455344699266


In [30]:
lassolars_cv = linear_model.LassoLarsCV(cv=50)
lassolars_cv.fit(X_train, y_train[:,1])

in_y_pred = lassolars_cv.predict(X_train)
out_y_pred = lassolars_cv.predict(X_test)

print(f'In-sample RMSE, R2: {np.sqrt(mean_squared_error(y_train[:,1], in_y_pred)):.2f}, {r2_score(y_train[:,1], in_y_pred)}')
print(f'Out-sample RMSE, R2: {np.sqrt(mean_squared_error(y_test[:,1], out_y_pred)):.2f}, {r2_score(y_test[:,1], out_y_pred)}')

In-sample RMSE, R2: 1.88, 0.10273262146209139
Out-sample RMSE, R2: 1.79, 0.07553288376438494


In [31]:
lasso = linear_model.Lasso(random_state=42)
lasso.fit(X_train, y_train[:,1])

in_y_pred = lasso.predict(X_train)
out_y_pred = lasso.predict(X_test)

print(f'In-sample RMSE, R2: {np.sqrt(mean_squared_error(y_train[:,1], in_y_pred)):.2f}, {r2_score(y_train[:,1], in_y_pred)}')
print(f'Out-sample RMSE, R2: {np.sqrt(mean_squared_error(y_test[:,1], out_y_pred)):.2f}, {r2_score(y_test[:,1], out_y_pred)}')

In-sample RMSE, R2: 1.92, 0.0674658247083868
Out-sample RMSE, R2: 1.78, 0.09056467133005774


In [46]:
joblib.dump(lasso, '/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Modelling/final_models/03_domain_engagement/collection_ratio.pkl')

['/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Modelling/final_models/03_domain_engagement/collection_ratio.pkl']

### Number of Backers

In [34]:
poisson = linear_model.PoissonRegressor()
poisson.fit(X_train, y_train[:,2])

in_y_pred = poisson.predict(X_train)
out_y_pred = poisson.predict(X_test)

print(f'In-sample RMSE, R2: {np.sqrt(mean_squared_error(y_train[:,2], in_y_pred)):.2f}, {r2_score(y_train[:,2], in_y_pred)}')
print(f'Out-sample RMSE, R2: {np.sqrt(mean_squared_error(y_test[:,2], out_y_pred)):.2f}, {r2_score(y_test[:,2], out_y_pred)}')

In-sample RMSE, R2: 1728.23, 0.3381106304125575
Out-sample RMSE, R2: 2641.30, 0.32335332849315834


In [45]:
joblib.dump(poisson, '/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Modelling/final_models/03_domain_engagement/num_backers.pkl')

['/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Modelling/final_models/03_domain_engagement/num_backers.pkl']

### Success

In [40]:
easy_ensemble = EasyEnsembleClassifier(random_state=42)
easy_ensemble.fit(X_train, y_train[:,0])

y_pred_in = easy_ensemble.predict(X_train)
y_pred_out = easy_ensemble.predict(X_test)

def get_class_wise_acc(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    return cm[0,0]/cm[0].sum(), cm[1,1]/cm[1].sum()

in_class_0_acc, in_class_1_acc = get_class_wise_acc(y_train[:,0], y_pred_in)
out_class_0_acc, out_class_1_acc = get_class_wise_acc(y_test[:,0], y_pred_out)

print(f'Overall In-sample Accuracy: {accuracy_score(y_train[:,0], y_pred_in)}')
print(f'Overall Out-sample Accuracy: {accuracy_score(y_test[:,0], y_pred_out)} \n')

print(f'In-sample Class 0 Accuracy: {in_class_0_acc}')
print(f'Out-sample Class 0 Accuracy: {out_class_0_acc} \n')

print(f'Out-sample Class 1 Accuracy: {out_class_1_acc}')
print(f'In-sample Class 1 Accuracy: {in_class_1_acc}')


Overall In-sample Accuracy: 0.6545064377682404
Overall Out-sample Accuracy: 0.6265060240963856 

In-sample Class 0 Accuracy: 1.0
Out-sample Class 0 Accuracy: 0.75 

Out-sample Class 1 Accuracy: 0.6133333333333333
In-sample Class 1 Accuracy: 0.6166666666666667


In [42]:
n_estimator_list = [75, 100, 500, 1000, 2000]

for n_estimators in n_estimator_list:
    balanced_rf = BalancedRandomForestClassifier(random_state=42, n_estimators=n_estimators)
    balanced_rf.fit(X_train, y_train[:,0])
    
    y_pred_in = balanced_rf.predict(X_train)
    y_pred_out = balanced_rf.predict(X_test)
    
    
    in_class_0_acc, in_class_1_acc = get_class_wise_acc(y_train[:,0], y_pred_in)
    out_class_0_acc, out_class_1_acc = get_class_wise_acc(y_test[:,0], y_pred_out)
    
    print(f'Number of Estimators: {n_estimators}')
    print(f'Overall In-sample Accuracy: {accuracy_score(y_train[:,0], y_pred_in)}')
    print(f'Overall Out-sample Accuracy: {accuracy_score(y_test[:,0], y_pred_out)} \n')
    
    print(f'In-sample Class 0 Accuracy: {in_class_0_acc}')
    print(f'Out-sample Class 0 Accuracy: {out_class_0_acc} \n')
    
    print(f'Out-sample Class 1 Accuracy: {out_class_1_acc}')
    print(f'In-sample Class 1 Accuracy: {in_class_1_acc}')
    print('-'*10)

Number of Estimators: 75
Overall In-sample Accuracy: 0.7188841201716738
Overall Out-sample Accuracy: 0.6626506024096386 

In-sample Class 0 Accuracy: 1.0
Out-sample Class 0 Accuracy: 0.5 

Out-sample Class 1 Accuracy: 0.68
In-sample Class 1 Accuracy: 0.6880952380952381
----------
Number of Estimators: 100
Overall In-sample Accuracy: 0.6952789699570815
Overall Out-sample Accuracy: 0.6987951807228916 

In-sample Class 0 Accuracy: 1.0
Out-sample Class 0 Accuracy: 0.5 

Out-sample Class 1 Accuracy: 0.72
In-sample Class 1 Accuracy: 0.6619047619047619
----------
Number of Estimators: 500
Overall In-sample Accuracy: 0.723175965665236
Overall Out-sample Accuracy: 0.6987951807228916 

In-sample Class 0 Accuracy: 1.0
Out-sample Class 0 Accuracy: 0.5 

Out-sample Class 1 Accuracy: 0.72
In-sample Class 1 Accuracy: 0.6928571428571428
----------
Number of Estimators: 1000
Overall In-sample Accuracy: 0.7317596566523605
Overall Out-sample Accuracy: 0.7108433734939759 

In-sample Class 0 Accuracy: 1.0


In [44]:
joblib.dump(easy_ensemble, '/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Modelling/final_models/03_domain_engagement/success.pkl')

['/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Modelling/final_models/03_domain_engagement/success.pkl']